In [83]:
import cv2
import numpy as np
import os
import glob

In [84]:
# show img for testing
def show_image_by_OpenCV(img):
    cv2.imshow('My Image', img )
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [85]:
def load_images_from_folder(folder):
    paths = glob.glob(os.path.join(folder,'*'))
    images = [cv2.imread(i) for i in paths]
    images = [cv2.resize(i, (i.shape[0]//10, i.shape[1]//10), interpolation=cv2.INTER_AREA) for i in images]
    return images

In [86]:
def nonmax_suppression(image):

    NONMAX_SUPPRESSION_KERNEL = 15
    STRIDE = 15
    
    result = np.zeros((image.shape))
    
    _image = np.lib.stride_tricks.sliding_window_view(image, NONMAX_SUPPRESSION_KERNEL, 1)[:,::STRIDE]
    _image = np.lib.stride_tricks.sliding_window_view(_image, NONMAX_SUPPRESSION_KERNEL, 0)[::STRIDE,:]
    # print(_image.shape)
    _image = np.reshape(_image,(*_image.shape[:2],-1))
    _image = np.argmax(_image, axis=2)

    mg = np.mgrid[0:_image.shape[0],0:_image.shape[1]]

    # y offset: mg[0], x offset: mg[1]
    index_y = _image // NONMAX_SUPPRESSION_KERNEL + mg[0] * STRIDE 
    index_x = _image % NONMAX_SUPPRESSION_KERNEL + mg[1] * STRIDE 
    index_x, index_y = index_x.flatten(), index_y.flatten()


    for i, j in zip(index_y, index_x):
        result[i][j]=image[i][j]  
        
    return  result

In [87]:
def detect_Harris_corner(image):
    # parameters
    KERNEL_SIZE = 11
    THRESHOLD = 100
    K = 0.04

    #  Compute x and y derivatives of image.
    # rgb to grayscale
    Gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    # gaussion : to do small smooth
    Gau_img = cv2.GaussianBlur(Gray_img, (5,5),5)
    # gradient
    I_y = np.gradient(Gau_img, axis=0) 
    I_x = np.gradient(Gau_img, axis=1) 


    # Compute products of derivates at each pixel.
    I_xx = I_x * I_x
    I_yy = I_y * I_y
    I_xy = I_x * I_y

    # Compute the sums of the products of derivates at each pixel.
    # weight : use gaussian
    S_xx = cv2.GaussianBlur(I_xx, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    S_yy = cv2.GaussianBlur(I_yy, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    S_xy = cv2.GaussianBlur(I_xy, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    
    # Define the matrix M.
    # | S_xx  S_xy |
    # | S_xy  S_yy |

    # Compute the response of the detector at each pixel
    detM = S_xx * S_yy - S_xy * S_xy
    traceM = S_xx + S_yy
    R = detM - K * (traceM * traceM)

    # Nonmax Suppression
    R = nonmax_suppression(R)
    
    # Threshold on value of R
    R[R < THRESHOLD] = 0
    R[R > 0] = 255

    
    # 濾邊界
    R[:20 , :] = 0
    R[: ,:20 ] = 0
    R[-20:, :] = 0
    R[:,-20: ] = 0
    # return mask
    # print(R.shape)
    return R
        

In [88]:
def feature_descriptor(feature_mask, image):

    Gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    gau_img = cv2.GaussianBlur(Gray_img, (3,3), 3)
    sobel_y = cv2.Sobel(gau_img, cv2.CV_64F, 0, 1)
    sobel_x = cv2.Sobel(gau_img, cv2.CV_64F, 1, 0)
    _, angle = cv2.cartToPolar(sobel_x, sobel_y, angleInDegrees=True)
    # print(angle.dtype)
    DESCRIPTOR_SIZE = 40
    KERNEL = 5

    # descriptor size: 40*40
    feature_point_y, feature_point_x = np.nonzero(feature_mask)
    index_to_get = [KERNEL//2 + i * KERNEL for i in range(DESCRIPTOR_SIZE//KERNEL)]
    # 5*5 sum/mean -> 8*8(descriptor matrix)

    descriptors = []
    positions = []
    for y, x in zip(feature_point_y, feature_point_x):
        y, x = int(y), int(x)
        rot_M = cv2.getRotationMatrix2D((x,y),angle[y,x],1) # (旋轉中心),旋轉角度,縮放比例
        img_rotate = cv2.warpAffine(gau_img,rot_M,(gau_img.shape[1],gau_img.shape[0]), flags = cv2.INTER_NEAREST)

        # 取feature_point-20~+19之間的值，做boxfilter
        big_matrix = img_rotate[y-DESCRIPTOR_SIZE//2:y+DESCRIPTOR_SIZE//2, x-DESCRIPTOR_SIZE//2:x+DESCRIPTOR_SIZE//2]
        big_matrix = cv2.boxFilter(big_matrix, -1, (KERNEL, KERNEL))
        # print(big_matrix.dtype)
        # size = (8, 8)
        descriptor = np.zeros((DESCRIPTOR_SIZE//KERNEL, DESCRIPTOR_SIZE//KERNEL))
        for i in range(DESCRIPTOR_SIZE//KERNEL):
            for j in range(DESCRIPTOR_SIZE//KERNEL):
                descriptor[i][j] = big_matrix[index_to_get[i]][index_to_get[j]]
        descriptor = (descriptor - np.mean(descriptor)) / np.std(descriptor)
        descriptors.append(descriptor)
        positions.append([y, x])

    return np.array(descriptors), np.array(positions)

In [89]:
def find_match_poing(descriptors_1, positions_1, descriptors_2, positions_2, LOW_THERESHOLD=2.0):
    match_point = []
    # match_dis = []
    
    for ds1, ps1 in zip(descriptors_1, positions_1):
        min_dis = 99999
        min_pos = [-1, -1]

        for ds2, ps2 in zip(descriptors_2, positions_2):
            dis = np.linalg.norm(ds1 - ds2)

            if(dis<min_dis):
                min_dis = dis
                min_pos = np.array([*ps1, *ps2])
                
        if(min_dis<LOW_THERESHOLD):
            match_point.append(min_pos)
            # match_dis.append(min_dis)
    return match_point

In [90]:
def matching(descriptors_1, positions_1, descriptors_2, positions_2):
    # print('find_match_poing1')
    match_point_to_1 = find_match_poing(descriptors_1, positions_1, descriptors_2, positions_2)
    
    # print('find_match_poing2')
    match_point_to_2 = find_match_poing(descriptors_2, positions_2, descriptors_1, positions_1)

    # print('merge')
    match_point = []
    for match_1 in match_point_to_1:
        for match_2 in match_point_to_2:
            dis_1 = np.linalg.norm(match_1[:2]-match_2[-2:])
            dis_2 = np.linalg.norm(match_1[-2:]-match_2[:2])
            
            if(dis_1+dis_2==0):
                match_point.append(match_1)
                
    return np.array(match_point)
    
    # [pos1_y, pos1_x, pos2_y, pos2_x]
    

In [91]:
def draw_point(img, R, output, outputName, r, g, b):
    for i in range(R.shape[0]):
        for j in range(R.shape[1]):
            if (R[i][j]>1):
                img = cv2.circle(img, (j, i), 2, (b, g, r), 2)
    if(output):
        cv2.imwrite(f"{outputName}.jpg", img)

In [92]:
def output_matching_image(images):
    print('Detecting Harris corner.')
    imgs_detect_Harris_corner = [detect_Harris_corner(i) for i in images]
    
    print('Genetating images feature descriptor.')
    for i, j in zip(images, imgs_detect_Harris_corner):
        draw_point(i, j, 0, '', 255, 0, 0) 
    dsps = [feature_descriptor(i, j)for i,j in zip(imgs_detect_Harris_corner, images)]
    match_points = []

    print('Matching descriptor.')
    for i in range(1, len(images)):
        match_points.append(matching(dsps[i-1][0], dsps[i-1][1], dsps[i][0], dsps[i][1]))
    return match_points

In [93]:
def draw_match_line(imgs, match_points):
    for i in range(len(match_points)): 
        img_concat = np.concatenate((imgs[i], imgs[i+1]), axis=1)
        img_w = imgs[i].shape[1]
        for j in match_points[i]:
            img_concat = cv2.line(img_concat, (j[1], j[0]), (j[3]+img_w, j[2]), (255,0,0), 1)

        cv2.imwrite(f'result{i}_{i+1}.jpg', img_concat)

In [94]:
# load images
images = load_images_from_folder('img/gym/2-2')

# return match point 
match_points = output_matching_image(images)

draw_match_line(images, match_points)

Detecting Harris corner.
Genetating images feature descriptor.
Matching descriptor.


In [95]:
'''
a = load_images_from_folder('img/test')
R = detect_Harris_corner(a[0])
R2 = detect_Harris_corner(a[1])

for i in range(R.shape[0]):
    for j in range(R.shape[1]):
        if (R[i][j]>1):
            a[0] = cv2.circle(a[0], (j, i), 2, (0, 0, 255), 2)

cv2.imwrite("R1.jpg", a[0])

for i in range(R2.shape[0]):
    for j in range(R2.shape[1]):
        if (R2[i][j]>1):
            a[1] = cv2.circle(a[1], (j, i), 2, (0, 0, 255), 2)

cv2.imwrite("R2.jpg", a[1])
ds1, ps1 = feature_descriptor(R, a[0])
ds2, ps2 = feature_descriptor(R2, a[1])
mpm = matching(ds1, ps1, ds2, ps2)
img_concat = np.concatenate((a[0], a[1]), axis=1)

img_w = a[0].shape[1]

for i in range(len(mpm)):
    img_concat = cv2.line(img_concat, (mpm[i][1], mpm[i][0]), (mpm[i][3]+img_w, mpm[i][2]), (255,0,0), 1)

cv2.imwrite("result04.jpg", img_concat)
'''

'\na = load_images_from_folder(\'img/test\')\nR = detect_Harris_corner(a[0])\nR2 = detect_Harris_corner(a[1])\n\nfor i in range(R.shape[0]):\n    for j in range(R.shape[1]):\n        if (R[i][j]>1):\n            a[0] = cv2.circle(a[0], (j, i), 2, (0, 0, 255), 2)\n\ncv2.imwrite("R1.jpg", a[0])\n\nfor i in range(R2.shape[0]):\n    for j in range(R2.shape[1]):\n        if (R2[i][j]>1):\n            a[1] = cv2.circle(a[1], (j, i), 2, (0, 0, 255), 2)\n\ncv2.imwrite("R2.jpg", a[1])\nds1, ps1 = feature_descriptor(R, a[0])\nds2, ps2 = feature_descriptor(R2, a[1])\nmpm = matching(ds1, ps1, ds2, ps2)\nimg_concat = np.concatenate((a[0], a[1]), axis=1)\n\nimg_w = a[0].shape[1]\n\nfor i in range(len(mpm)):\n    img_concat = cv2.line(img_concat, (mpm[i][1], mpm[i][0]), (mpm[i][3]+img_w, mpm[i][2]), (255,0,0), 1)\n\ncv2.imwrite("result04.jpg", img_concat)\n'